In [1]:
import time
import torch
from vmas import make_env
from vmas.simulator.core import Agent
from vmas.simulator.scenario import BaseScenario
from typing import Union
from moviepy.editor import ImageSequenceClip
from IPython.display import HTML, display as ipython_display
import numpy as np
import matplotlib.pyplot as plt
from gym.spaces import Discrete 

class QLearningAgent:
    def __init__(self, env, agent_id, alpha=0.1, gamma=0.99, epsilon=0.1, communication_weight=0.5):
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate
        self.q_table = {}
        self.env = env
        self.agent_id = agent_id
        self.communication_weight = communication_weight  # Weight parameter for incorporating messages
        self.action_count = 0  # Initialize action count to 0

    def get_action(self, agent, env, agent_id, agent_obs):
        # Increment action count whenever get_action is called
        self.action_count += 1

        obs_tuple = tuple(agent_obs.cpu().numpy().flatten())
        obs_tuple_agent = obs_tuple[0:5]
        if not agent.silent:
            obs_tuple_comm = obs_tuple[7]
        
        if obs_tuple_agent not in self.q_table:
            self.q_table[obs_tuple_agent] = np.zeros(self.env.action_space[self.agent_id].n)
        
        if np.random.rand() < self.epsilon:
            physical_action = env.get_random_action(agent)
        else:
            physical_action = (np.argmax(self.q_table[obs_tuple_agent]),)  # Exploit: best known action
        
        if agent.silent:
            action = torch.tensor(physical_action, device=self.env.device)
        else:
            physical_action_tensor = torch.tensor(physical_action, device=self.env.device)
            comm_action_tensor = torch.tensor([obs_tuple_comm], device=self.env.device)  # Adding a dimension to obs_tuple_comm
            
            if agent_id == 1:
                physical_action_tensor = torch.tensor(comm_action_tensor, device=self.env.device)

            # comm_action_tensor = []           
            action = torch.cat([physical_action_tensor, comm_action_tensor], dim=0)

        return action.unsqueeze(0)

    

    def update_q_table(self, obs, action, reward, next_obs):
        obs_tuple = tuple(obs.cpu().numpy().flatten())
        next_obs_tuple = tuple(next_obs.cpu().numpy().flatten())
        
        if isinstance(self.env.action_space[self.agent_id], Discrete):
            action_space_size = self.env.action_space[self.agent_id].n
        else:
            raise ValueError("This Q-learning implementation requires a discrete action space.")

        if obs_tuple not in self.q_table:
            self.q_table[obs_tuple] = np.zeros(action_space_size)
        if next_obs_tuple not in self.q_table:
            self.q_table[next_obs_tuple] = np.zeros(action_space_size)

        best_next_action = np.argmax(self.q_table[next_obs_tuple])
        td_target = reward + self.gamma * self.q_table[next_obs_tuple][best_next_action]

        if isinstance(action, torch.Tensor):
            action = action[0, 0].item()  # Extract the first element from the 2D tensor

        action = int(action)  # Explicitly convert to integer

        td_error = td_target - self.q_table[obs_tuple][action]
        self.q_table[obs_tuple][action] += self.alpha * td_error




class VMASEnvRunner:
    def __init__(
        self,
        render: bool,
        num_envs: int,
        num_episodes: int,
        max_steps_per_episode: int,
        device: str,
        scenario: Union[str, BaseScenario],
        continuous_actions: bool,
        random_action: bool,
        n_agents: int,
        obs_discrete: bool = False,
        **kwargs
    ):
        self.render = render
        self.num_envs = num_envs
        self.num_episodes = num_episodes
        self.max_steps_per_episode = max_steps_per_episode
        self.device = device
        self.scenario = scenario
        self.continuous_actions = continuous_actions
        self.random_action = random_action
        self.obs_discrete = obs_discrete
        self.kwargs = kwargs
        self.frame_list = []  
        self.q_learning_agents = []
        self.rewards_history = []  
        self.action_counts = {i: {} for i in range(n_agents)}  

    def discretize(self, data, bins):
        bins = np.array(bins)
        if np.isscalar(data):
            bin_indices = np.digitize([data], bins)[0]
            bin_values = bins[bin_indices - 1] if bin_indices > 0 else bins[0]
            return bin_indices, bin_values
        else:
            bin_indices = np.digitize(data, bins)
            bin_values = [bins[index - 1] if index > 0 else bins[0] for index in bin_indices]
            return bin_indices, bin_values

    def discretize_tensor_slice(self, tensor_slice, bins):
        indices = []
        values = []
        
        # Check if the tensor_slice is 1D or 2D
        if tensor_slice.dim() == 1:
            for item in tensor_slice:
                bin_idx, bin_val = self.discretize(item.item(), bins)
                indices.append(bin_idx)
                values.append(bin_val)
        elif tensor_slice.dim() == 2:
            for row in tensor_slice:
                row_indices = []
                row_values = []
                for item in row:
                    bin_idx, bin_val = self.discretize(item.item(), bins)
                    row_indices.append(bin_idx)
                    row_values.append(bin_val)
                indices.append(row_indices)
                values.append(row_values)
        else:
            raise ValueError("tensor_slice must be either 1D or 2D")

        return torch.tensor(indices, device=tensor_slice.device), torch.tensor(values, device=tensor_slice.device)

    def _get_deterministic_obs(self, env, observation):
        pos_bins = np.linspace(-1, 1, num=21)
        vel_bins = np.linspace(-1, 1, num=21)
        lidar_bins = np.linspace(0, 1, num=21)

        # print(f"observation passed value: {observation}")
        pos = torch.tensor(observation[0:2], device=self.device)
        vel = observation[2:4]
        goal_pose = observation[4:6]
        sensor_data = observation[6:]
        comms_data = observation[6:]
        # print(f"pos: {pos}")

        discrete_pos_indices, discrete_pos_values = self.discretize_tensor_slice(pos, pos_bins)
        discrete_vel_indices, discrete_vel_values = self.discretize_tensor_slice(vel, vel_bins)
        discrete_goal_pose_indices, discrete_goal_pose_values = self.discretize_tensor_slice(goal_pose, pos_bins)
        discrete_sensor_data_indices, discrete_sensor_data_values = self.discretize_tensor_slice(sensor_data, lidar_bins)

        concatenated_tensor_values = torch.cat(
            [discrete_pos_values, discrete_vel_values, discrete_goal_pose_values, discrete_sensor_data_values, comms_data],
            dim=0
        )

        return concatenated_tensor_values

    def _get_deterministic_action(self, agent: Agent, env, agent_id, agent_obs):
        if self.continuous_actions:
            if agent.silent:
                action = torch.tensor([[-1, 0.5]], device=env.device)
            else:
                if agent_id == 0:
                    action = torch.tensor([[-1, 0.5, 2]], device=env.device)
                else:
                    action = torch.tensor([[-1, 0.5, 1]], device=env.device)
        else:
            # action = self.q_learning_agents[agent_id].get_action(agent,env, agent_id, agent_obs)
            physical_action = torch.tensor([2], device=env.device)

            communication_problem_action = torch.tensor([[3, 5, 1, 4, 6, 8]], device=env.device)

            num_rows, num_cols = communication_problem_action.shape
            num_values = len(physical_action)
            padding_needed = num_cols - num_values

            if padding_needed > 0:
                physical_action_padded = torch.cat((physical_action, torch.zeros(padding_needed, device=env.device)))
            else:
                physical_action_padded = physical_action[:num_cols]

            # Create communication_solution_action with the same shape as communication_problem_action
            communication_solution_action = physical_action_padded.repeat(num_rows, 1)
            # print(communication_solution_action)  # Output: torch.Size([1, 6])

            communication_action = torch.cat((communication_problem_action, communication_solution_action), dim=0)
            # print(communication_action.shape)  # Output: torch.Size([1, 6])

            # action = (physical_action, communication_action)
            
            # action = torch.tensor([[[[2, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0]], [[3, 3, 2, 2, 1, 1], [2, 0, 0, 0, 0, 0]]]], device=env.device)
            action = torch.tensor([[[2, 0, 0, 0, 0, 0, 0], [-0.56,  0.33,  0.0000,  0.0000, -0.1, 0.25,  2]]], device=env.device)
            


        # return action.clone()
        return action # ex result: (tensor([2]), tensor([[3., 5., 1., 4., 6., 8.], [2., 0., 0., 0., 0., 0.]]))

    def generate_gif(self, scenario_name):
        fps = 25
        clip = ImageSequenceClip(self.frame_list, fps=fps)
        clip.write_gif(f'{scenario_name}.gif', fps=fps)
        return HTML(f'<img src="{scenario_name}.gif">')

    def plot_action_distribution(self):
        num_agents = len(self.action_counts)

        for agent_id, counts in self.action_counts.items():
            unique_actions, action_counts = np.unique(list(counts.values()), return_counts=True)
            action_dict = dict(zip(unique_actions, action_counts))
            plt.bar(action_dict.keys(), action_dict.values(), label=f'Agent {agent_id}', alpha=0.7)

        plt.xlabel('Action')
        plt.ylabel('Frequency')
        plt.title('Action Distribution for Each Agent')
        plt.legend()
        plt.show()

    def run_vmas_env(self):
        scenario_name = self.scenario if isinstance(self.scenario, str) else self.scenario.__class__.__name__

        env = make_env(
            scenario=self.scenario,
            num_envs=self.num_envs,
            device=self.device,
            continuous_actions=self.continuous_actions,
            seed=0,
            **self.kwargs
        )

        # Initialize Q-learning agents
        for agent_id, agent in enumerate(env.agents):
            self.q_learning_agents.append(QLearningAgent(env, agent_id, communication_weight=0.5))

        init_time = time.time()
        total_steps = 0
        same_action_count = 0  # Counter for same actions taken by both agents
        total_action_count = 0  # Total number of actions taken

        for e in range(self.num_episodes):
            print(f"Episode {e}")
            obs = env.reset()
            print(f"observation reset result: {obs}")
            done = [False] * self.num_envs
            step = 0
            while not all(done) and step < self.max_steps_per_episode:
                step += 1
                total_steps += 1
                print(f"Step {step} of Episode {e}")

                actions = []
                for i, agent in enumerate(env.agents):
                    if self.obs_discrete:
                        discrete_obs = self._get_deterministic_obs(env, obs[i])
                        obs[i] = discrete_obs

                    if self.random_action:
                        action = env.get_random_action(agent)
                    else:
                        action = self._get_deterministic_action(agent, env, i, obs[i])
                    
                    # print(f"Action agent {i}: {action}")
                    # physical_action = action[0, 0, 0].unsqueeze(0)
                    # print(f"tes physical action: {physical_action}")
                    actions.append(action)
                    
                    # print (f"action length: {len(actions)}, n agents: {env.n_agents}")
                    # Track actions taken by both agents
                    # if i == 1 and actions[0][0, 0].item() == actions[1][0, 0].item():
                    #     same_action_count += 1
                    # total_action_count += 1

                    # self.action_counts[i][step] = action[0, 0].item()

                next_obs, rews, dones, info = env.step(actions)
                # print(f"observation after actions: {next_obs}")

                done = [done or d for done, d in zip(done, dones)]

                if len(self.rewards_history) <= step:
                    self.rewards_history.append([0] * len(env.agents))
                for i, rew in enumerate(rews):
                    self.rewards_history[step-1][i] = rew.item()

                for i, agent in enumerate(env.agents):
                    self.q_learning_agents[i].update_q_table(obs[i], actions[i], rews[i], next_obs[i])

                obs = next_obs

                if self.render:
                    frame = env.render(
                        mode="rgb_array",
                        agent_index_focus=None,
                    )
                    self.frame_list.append(frame)

        total_time = time.time() - init_time
        print(
            f"It took: {total_time}s for {total_steps} steps across {self.num_episodes} episodes of {self.num_envs} parallel environments on device {self.device} "
            f"for {scenario_name} scenario."
        )

        # Calculate percentage of times both agents take the same action
        # same_action_percentage = (same_action_count / total_action_count) * 100
        # print(f"Percentage of times Agent 1 and Agent 2 take the same action: {same_action_percentage:.2f}%")

        # self.plot_action_distribution()


if __name__ == "__main__":
    scenario_name = "navigation_comm"

    env_runner = VMASEnvRunner(
        render=True,
        num_envs=1,
        num_episodes=1,
        max_steps_per_episode=2,
        device="cpu",
        scenario=scenario_name,
        continuous_actions=False,
        random_action=False,
        n_agents=2,
        obs_discrete=True,
        agents_with_same_goal=2,
        collisions=False,
        shared_rew=False,
    )
    env_runner.run_vmas_env()
    ipython_display(env_runner.generate_gif(scenario_name))



/home/ardie85/PHD/Research/code/.venv/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


Episode 0
observation reset result: [tensor([-0.0887,  0.2646,  0.0000,  0.0000, -0.9553, -0.6623,  0.0000]), tensor([-0.3022, -0.1966,  0.0000,  0.0000, -0.9553, -0.6623,  0.0000])]
Step 1 of Episode 0
physical actionnneeeee: tensor([2.])
physical actionnneeeee size: torch.Size([1])
communication actionnneeeee: tensor([-0.5600,  0.3300,  0.0000,  0.0000, -0.1000,  0.2500,  2.0000])
communication actionnneeeee size: torch.Size([7])
physical actionnneeeee: tensor([2.])
physical actionnneeeee size: torch.Size([1])
communication actionnneeeee: tensor([-0.5600,  0.3300,  0.0000,  0.0000, -0.1000,  0.2500,  2.0000])
communication actionnneeeee size: torch.Size([7])


RuntimeError: a Tensor with 7 elements cannot be converted to Scalar